In [ ]:
import math 
import os
import json
import csv
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from skimage.feature import graycoprops, graycomatrix, local_binary_pattern, hog
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage import data, exposure, io
from sklearn.preprocessing import StandardScaler

In [ ]:
json_file = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\module_metadata.json"
avg_glcmfeatures_nopreprocess_csv = r"C:\Users\mdrsp\Desktop\School\For thesis\csv_files\AVG_glcm_features_no_preprocessing.csv"
glcmfeatures_nopreprocess_csv = r"C:\Users\mdrsp\Desktop\School\For thesis\csv_files\glcm_features_no_preprocessing.csv"

sample_img_path = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\images\0.jpg" # path to sample image for testing the code

path_2_images = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules"

pre processing and feature extraction functions takes in array input and not the file path!!

In [ ]:
# some simple but important functions

# takes in a path and returns the np array
def path2img(path):
    img = io.imread(path,as_gray=True)
    return img

In [ ]:
# Pre processing.
# function includes grascale conversion, Histogram equalization, adn noise filering
# gabor_filtering is used for noise filtering. 

def pre_processing(img):
    hist_img = cv.equalizeHist(img)
    gabor_filter = create_gaborfilter()
    filtered_image = filtering(hist_img, gabor_filter) 
    return filtered_image

def create_gaborfilter():
    filters = []
    num = 16
    sigma = 1.0
    lambd = 30
    gamma = 0.25
    psi = 0
    for theta in np.arange(0, np.pi, np.pi/num):
        kern = cv.getGaborKernel((3,3), sigma, theta, lambd, gamma, psi, ktype=cv.CV_64F)
        kern /= 1.0*kern.sum()
        filters.append(kern)
    return filters

def filtering(img, filters):  
    newimage = np.zeros_like(img)
    depth = -1
    for kernels in filters:
        image_filter = cv.filter2D(img, depth, kernels) 
        np.maximum(newimage, image_filter, newimage)
    return newimage



In [ ]:
def apply_gaussian_filter(image, kernel_size=(5, 5), sigma=0):
    return cv.GaussianBlur(image, kernel_size, sigma)

In [ ]:
# Feature extraraction functions, all return np.ndarrays

# Gray-Level Co-Occurrence Matrix
def glcm_features(img):    
    # Calculate the GLCM
    #features = ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'correlation', 'energy']
    features = ['dissimilarity', 'homogeneity', 'correlation']
    glcm = graycomatrix(img, distances=[1,2,3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    feature_vector = []
    for feature in features:
        feature_value = graycoprops(glcm, feature)
        feature_vector.append(feature_value)
    return feature_vector


# Local binary Pattern
def lbp_features(img):
    from skimage.feature import local_binary_pattern
    radius = 3
    n_points = 8 * radius
    lbp_value = local_binary_pattern(img, n_points,radius)
    return lbp_value


# Histogram of oriented gradients
def hog_features(img):
    # Calculate the HOG features
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1), visualize=True)

    return fd


In [ ]:
sample_img = path2img(sample_img_path)
print(sample_img)

In [ ]:
np.array(glcm_features(sample_img))


In [ ]:
lbp_features(sample_img)

In [ ]:
hog_features(sample_img)

In [ ]:
# this code just create a new json file with updated classes
file = open(json_file)
data = json.load(file)
new_data = dict()

for key,value in data.items():
    current_img = value['image_filepath']
    anomaly = value['anomaly_class']
    if anomaly in ["Cell", "Cell-Multi", "Hot-Spot-Multi"]:
        anomaly = "Hot-Spot"
    elif anomaly =="Diode-Multi":
        anomaly = "Diode"
    elif anomaly == "Vegetation" or anomaly == "Soiling":
        anomaly = "Shadowing"
    
    new_data[key] = {'image_filepath':current_img, 'anomaly_class':anomaly }
    

json_object = json.dumps(new_data)
with open(r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\module_metadata_update.json", 'w') as f:
    f.write(json_object)

In [ ]:
# this code just create a new json file with updated classes
file = open(json_file)
data = json.load(file)
new_data = dict()

for key,value in data.items():
    current_img = value['image_filepath']
    anomaly = value['anomaly_class']
    if anomaly in ["Cell", "Cell-Multi", "Hot-Spot-Multi"]:
        anomaly = "Hot-Spot"
    elif anomaly =="Diode-Multi":
        anomaly = "Diode"
    elif anomaly == "Vegetation" or anomaly == "Soiling":
        anomaly = "Shadowing"
    
    new_data[key] = {'image_filepath':current_img, 'anomaly_class':anomaly }
    

json_object = json.dumps(new_data)
with open(r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\module_metadata_update.json", 'w') as f:
    f.write(json_object)

In [15]:
updated_json = r"C:\Users\mdrsp\Desktop\School\For thesis\spv_images\InfraredSolarModules\module_metadata_update.json"
file = open(updated_json)
data = json.load(file)

images = []
labels = []

os.chdir(path_2_images)
for key,value in data.items():
    current_img_path = value['image_filepath']
    anomaly = value['anomaly_class']

    img = path2img(current_img_path)
    #img = apply_gaussian_filter(img)
   
    lbp_value = np.ravel(lbp_features(img))
    glcm_value = np.ravel(np.array(glcm_features(img)))
    hog_value = np.ravel(hog_features(img))   

    feature_vector = np.concatenate((glcm_value,lbp_value, hog_value))
    feature_vector = np.ravel(feature_vector)
       
    images.append(feature_vector)
    labels.append(anomaly)  

    

In [16]:
# Convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)


In [17]:
#normalize images array
scaler = StandardScaler()
images = scaler.fit_transform(images)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Train an SVM classifier
clf = SVC(kernel='poly', C=1, random_state=42)
clf.fit(X_train, y_train)

# Test the SVM classifier
accuracy = clf.score(X_test, y_test)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.669


In [ ]:
prediction = clf.predict(X_test)
print(classification_report(prediction, y_test))